## Dependencies

In [ ]:
!pip install weaviate-client

## Configuration

In [ ]:
import weaviate

# Connect to your local Weaviate instance deployed with Docker
client = weaviate.connect_to_local()

# Option 2
# Connect to your Weaviate Client Service cluster
# client = weaviate.connect_to_wcs(
#     cluster_id="WCS-CLUSTER-ID", # Replace with your WCS cluster ID
#     auth_credentials=weaviate.AuthApiKey(
#       api_key="WCS-API-KEY" # Replace with your WCS API KEY
#     ),
# )

client.is_ready()

## Create a collection
> Collection stores your data and vector embeddings.

In [ ]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "MyCollection", and then you need to re-import it again.
import weaviate.classes.config as wc

# Delete the collection if it already exists
if (client.collections.exists("MyCollection")):
    client.collections.delete("MyCollection")

client.collections.create(
    name="MyCollection",

    vectorizer_config=wc.Configure.Vectorizer.none(), # No vectorizer needed

    vector_index_config=wc.Configure.VectorIndex.hnsw(
        distance_metric=wc.VectorDistances.COSINE # select prefered distance metric
    ),

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="title", data_type=wc.DataType.TEXT), 
        wc.Property(name="foo", data_type=wc.DataType.INT),
    ]
)

print("Successfully created collection: MyCollection.")

## Import the Data

In [ ]:
sample_data = [
   wc.DataObject(
      properties={
         "title": "First Object",
         "foo": 99, 
      },
      vector=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
   ),
   wc.DataObject(
      properties={
         "title": "Second Object",
         "foo": 77,
      },
      vector=[0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
   ),
   wc.DataObject(
      properties={
         "title": "Third Object",
         "foo": 55,
      },
      vector=[0.3, 0.1, -0.1, -0.3, -0.5, -0.7]
   ),
   wc.DataObject(
      properties={
         "title": "Fourth Object",
         "foo": 33,
      },
      vector=[0.4, 0.41, 0.42, 0.43, 0.44, 0.45]
   ),
   wc.DataObject(
      properties={
         "title": "Fifth Object",
         "foo": 11,
      },
      vector=[0.5, 0.5, 0, 0, 0, 0]
   ),
]

In [ ]:
# Get a collection object for "MyCollection"
my_collection = client.collections.get("MyCollection")
my_collection.data.insert_many(sample_data)

print("Data import complete")

Quick check to see if all objects are in.
Let's use [meta count](https://weaviate.io/developers/weaviate/search/aggregate#retrieve-a-meta-property).

In [ ]:
# Check number of objects
my_collection.aggregate.over_all()

## Query Weaviate: Vector Search (vector embeddings)

Available types of queries you can run when working with vector embeddings (without modules) in **Weaviate**:

1. [near_vector](https://weaviate.io/developers/weaviate/search/similarity#search-with-a-vector)

2. [near_object](https://weaviate.io/developers/weaviate/search/similarity#search-with-an-existing-object)

### nearVector Example
**First example** - Search Weaviate with a vector embedding, and return title property.

See [the docs](https://weaviate.io/developers/weaviate/search/similarity#search-with-a-vector) for more.

In [ ]:
response = my_collection.query.near_vector(
    near_vector=[-0.012, 0.021, -0.23, -0.42, 0.5, 0.5],
    limit=2,
)

for item in response.objects:
    print(item.uuid)
    print(item.properties, "\n")

**Second example** - The same search query, but this time also return `distance`, and `vector`.

In [ ]:
import weaviate.classes.query as wq

response = my_collection.query.near_vector(
    near_vector=[-0.012, 0.021, -0.23, -0.42, 0.5, 0.5],
    include_vector=True,
    return_metadata=wq.MetadataQuery(distance=True),
    limit=2,
)

for item in response.objects:
    print(item.properties)
    print(item.metadata.distance)
    print(item.vector, "\n")

**Third example** – Same vector query, but this time we will filter on "foo" (which should be greater than 44). Also, let's return "title" and "foo".

See [the docs](https://weaviate.io/developers/weaviate/search/filters#filter-with-one-condition) for more.

In [ ]:
import weaviate.classes.query as wq

response = my_collection.query.near_vector(
    near_vector=[-0.012, 0.021, -0.23, -0.42, 0.5, 0.5],
    return_metadata=wq.MetadataQuery(distance=True),
    filters=wq.Filter.by_property("foo").greater_than(44),
    limit=2,
)

for item in response.objects:
    print(item.properties)
    print(item.metadata.distance, "\n")

### nearObject Example

Weaviate also allows you to search for similar objects.

See [the docs](https://weaviate.io/developers/weaviate/search/similarity#search-with-an-existing-object) for more.

**Fourth example** - 
Search through `MyCollection` for similar objects, by providing an id from the previous query. 

> Note #1: The id was taken from the query above <br/>
> The generated id for you might be different.

> Note #2: The first object returned is always itself.

In [ ]:
import weaviate.classes.query as wq

response = my_collection.query.near_object(
    near_object="989ab81d-6f9b-46f1-9be8-113eae84ed33",
    return_metadata=wq.MetadataQuery(distance=True),
    limit=3,
)

for item in response.objects:
    print(item.uuid)
    print(item.properties)
    print(item.metadata.distance, "\n")